In [341]:
import pandas as pd 
import dataparser 
from  math import log
import numpy as np
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
from dataparser import *

import folium

pd.set_option('display.max_columns', None)
data=pd.read_csv('_data.csv', index_col=1)

#если цена не в рублях то выбрасываем, цены на доллар и евро сильно меняются, непонятно какая цена в рублях (там всего 27 вариантов)
data=data[data['Цена'].str.contains('руб.')].reset_index()

#убираем всех не из Москвы
data=data[data['Адрес'].str.contains('Москва')].reset_index()

#заполняем пустые места для количества комнат, так как это студии, 1 комната
#фича "Тип" у всех одинаковая, дропаем
data=data.fillna({'Количество комнат':1})\
        .drop('Тип',axis=1)


In [342]:
data

,index,ID объявления,Unnamed: 0,Количество комнат,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Телефоны,Описание,Ремонт,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление
0,0,271271157,0,4,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79166369231,Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.00,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157
1,1,271634126,1,4,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79850608590,Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.50,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126
2,2,271173086,2,"4, Оба варианта",м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...","+79672086536, +79099269384","ID 36380: Шикарная 4-х км. квартира в ЖК ""Нов...",Евроремонт,NaN,NaN,На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.20,Пасс (1),NaN,https://www.cian.ru/rent/flat/271173086
3,3,272197456,3,"4, Оба варианта",м. Смоленская (3 мин пешком),"Москва, переулок Плотников, 21С1",170.0/95.0/17.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...","+79660342340, +79099269384",ID 31618: Эксклюзивное предложение. Современн...,Евроремонт,14-42-20-19,NaN,На улицу и двор,Совмещенный (3),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,3.20,Пасс (1),NaN,https://www.cian.ru/rent/flat/272197456
4,4,273614615,4,2,м. Арбатская (7 мин пешком),"Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",NaN,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",+79852432860,Лот 111542. Татьяна Лучкина.\n\nБонус агенту 5...,Евроремонт,20 18,NaN,На улицу и двор,Совмещенный (2),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",The Book,NaN,3.90,"Пасс (1), Груз (1)",Да,https://www.cian.ru/rent/flat/273614615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19709,23339,215565511,23363,1,м. Говорово (8 мин пешком),"Москва, Боровское шоссе, 2к7, ш. Боровское (3 ...",35.0/16.4/8.0,10/14,NaN,"42000.0 руб./ За месяц, Залог - 42000 руб., Ко...",+79154383401,Сдаётся светлая теплая квартира (студия) с бал...,Евроремонт,NaN,Балкон (1),NaN,Совмещенный (1),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Сти...",Мещерский лес,NaN,NaN,"Пасс (1), Груз (1)",NaN,https://www.cian.ru/rent/flat/215565511
19710,23340,274654844,23364,1,м. Солнцево (7 мин пешком),"Москва, Производственная улица, 8к1, ш. Боровс...",38.7/16.5/11.0,"5/18, Монолитный",NaN,"45000.0 руб./ За месяц, Залог - 45000 руб., Ко...",+79151173402,Сдам однокомнатную квартиру на длительный срок...,Евроремонт,NaN,Лоджия (1),Во двор,Совмещенный (1),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Сти...",Лучи,NaN,NaN,"Пасс (1), Груз (1)",NaN,https://www.cian.ru/rent/flat/274654844
19711,23341,268679909,23365,"2, Оба варианта",м. Солнцево (6 мин пешком),"Москва, Боровский проезд, 11",43.1,"5/5, Кирпичный",NaN,"50000.0 руб./ За месяц, Залог - 50000 руб., Ко...",+79653825192,Рассматриваются квартиранты только славяне и т...,Дизайнерский,NaN,Балкон (1),На улицу и двор,Совмещенный (1),Можно с детьми,"Мебель на кухне, Душевая кабина, Стиральная ма...",NaN,NaN,NaN,NaN,NaN,https://www.cian.ru/rent/flat/268679909
19712,23342,274807525,23366,2,м. Солнцево (11 мин 

In [343]:
def get_col_room(data):
    # Функция для извлечения количества комнат
    return data['Количество комнат'].astype(str).apply(lambda x: int(x[0]))
data['col_rooms'] = get_col_room(data)

In [346]:
def get_price(data):
    # Функция для извлечения цены
    return data['Цена'].apply(lambda x: float(splbac(x,'руб./')[0])).astype(int)

data['price'] = get_price(data)

In [348]:
def get_ceiling_height(data):
    # Функция для извлечения высоты потолков
    return data['Высота потолков, м'].astype(str).apply(lambda x: float(x.split()[0]))

data['ceiling_height'] = get_ceiling_height(data)

In [354]:
data = data[(data['ceiling_height'] < 7) | (np.isnan(data['ceiling_height']))]
data

,index,ID объявления,Unnamed: 0,Количество комнат,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Телефоны,Описание,Ремонт,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление,col_rooms,price,ceiling_height
0,0,271271157,0,4,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79166369231,Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.00,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157,4,500000,3.0
1,1,271634126,1,4,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79850608590,Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.50,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126,4,500000,3.5
2,2,271173086,2,"4, Оба варианта",м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...","+79672086536, +79099269384","ID 36380: Шикарная 4-х км. квартира в ЖК ""Нов...",Евроремонт,NaN,NaN,На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.20,Пасс (1),NaN,https://www.cian.ru/rent/flat/271173086,4,500000,3.2
3,3,272197456,3,"4, Оба варианта",м. Смоленская (3 мин пешком),"Москва, переулок Плотников, 21С1",170.0/95.0/17.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...","+79660342340, +79099269384",ID 31618: Эксклюзивное предложение. Современн...,Евроремонт,14-42-20-19,NaN,На улицу и двор,Совмещенный (3),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,3.20,Пасс (1),NaN,https://www.cian.ru/rent/flat/272197456,4,400000,3.2
4,4,273614615,4,2,м. Арбатская (7 мин пешком),"Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",NaN,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",+79852432860,Лот 111542. Татьяна Лучкина.\n\nБонус агенту 5...,Евроремонт,20 18,NaN,На улицу и двор,Совмещенный (2),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",The Book,NaN,3.90,"Пасс (1), Груз (1)",Да,https://www.cian.ru/rent/flat/273614615,2,225000,3.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19709,23339,215565511,23363,1,м. Говорово (8 мин пешком),"Москва, Боровское шоссе, 2к7, ш. Боровское (3 ...",35.0/16.4/8.0,10/14,NaN,"42000.0 руб./ За месяц, Залог - 42000 руб., Ко...",+79154383401,Сдаётся светлая теплая квартира (студия) с бал...,Евроремонт,NaN,Балкон (1),NaN,Совмещенный (1),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Сти...",Мещерский лес,NaN,NaN,"Пасс (1), Груз (1)",NaN,https://www.cian.ru/rent/flat/215565511,1,42000,3.0
19710,23340,274654844,23364,1,м. Солнцево (7 мин пешком),"Москва, Производственная улица, 8к1, ш. Боровс...",38.7/16.5/11.0,"5/18, Монолитный",NaN,"45000.0 руб./ За месяц, Залог - 45000 руб., Ко...",+79151173402,Сдам однокомнатную квартиру на длительный срок...,Евроремонт,NaN,Лоджия (1),Во двор,Совмещенный (1),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Сти...",Лучи,NaN,NaN,"Пасс (1), Груз (1)",NaN,https://www.cian.ru/rent/flat/274654844,1,45000,3.0
19711,23341,268679909,23365,"2, Оба варианта",м. Солнцево (6 мин пешком),"Москва, Боровский проезд, 11",43.1,"5/5, Кирпичный",NaN,"50000.0 руб./ За месяц, Залог - 50000 руб., Ко...",+79653825192,Рассматриваются квартиранты только славяне и т...,Дизайнерский,NaN,Балкон (1),На улицу и двор,Совмещенный (1),Можно с детьми,"Мебель на кухне, Душевая каби

In [355]:
data['ceiling_height'] = data['ceiling_height'].fillna(data['ceiling_height'].mean()).round(1)
data

/var/folders/l4/cnmt98l51vg5y92_6qkrhqq00000gn/T/ipykernel_41043/1889640643.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ceiling_height'] = data['ceiling_height'].fillna(data['ceiling_height'].mean()).round(1)


,index,ID объявления,Unnamed: 0,Количество комнат,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Телефоны,Описание,Ремонт,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление,col_rooms,price,ceiling_height
0,0,271271157,0,4,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79166369231,Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.00,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157,4,500000,3.0
1,1,271634126,1,4,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79850608590,Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.50,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126,4,500000,3.5
2,2,271173086,2,"4, Оба варианта",м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...","+79672086536, +79099269384","ID 36380: Шикарная 4-х км. квартира в ЖК ""Нов...",Евроремонт,NaN,NaN,На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.20,Пасс (1),NaN,https://www.cian.ru/rent/flat/271173086,4,500000,3.2
3,3,272197456,3,"4, Оба варианта",м. Смоленская (3 мин пешком),"Москва, переулок Плотников, 21С1",170.0/95.0/17.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...","+79660342340, +79099269384",ID 31618: Эксклюзивное предложение. Современн...,Евроремонт,14-42-20-19,NaN,На улицу и двор,Совмещенный (3),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,3.20,Пасс (1),NaN,https://www.cian.ru/rent/flat/272197456,4,400000,3.2
4,4,273614615,4,2,м. Арбатская (7 мин пешком),"Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",NaN,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",+79852432860,Лот 111542. Татьяна Лучкина.\n\nБонус агенту 5...,Евроремонт,20 18,NaN,На улицу и двор,Совмещенный (2),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",The Book,NaN,3.90,"Пасс (1), Груз (1)",Да,https://www.cian.ru/rent/flat/273614615,2,225000,3.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19709,23339,215565511,23363,1,м. Говорово (8 мин пешком),"Москва, Боровское шоссе, 2к7, ш. Боровское (3 ...",35.0/16.4/8.0,10/14,NaN,"42000.0 руб./ За месяц, Залог - 42000 руб., Ко...",+79154383401,Сдаётся светлая теплая квартира (студия) с бал...,Евроремонт,NaN,Балкон (1),NaN,Совмещенный (1),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Сти...",Мещерский лес,NaN,NaN,"Пасс (1), Груз (1)",NaN,https://www.cian.ru/rent/flat/215565511,1,42000,3.0
19710,23340,274654844,23364,1,м. Солнцево (7 мин пешком),"Москва, Производственная улица, 8к1, ш. Боровс...",38.7/16.5/11.0,"5/18, Монолитный",NaN,"45000.0 руб./ За месяц, Залог - 45000 руб., Ко...",+79151173402,Сдам однокомнатную квартиру на длительный срок...,Евроремонт,NaN,Лоджия (1),Во двор,Совмещенный (1),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Сти...",Лучи,NaN,NaN,"Пасс (1), Груз (1)",NaN,https://www.cian.ru/rent/flat/274654844,1,45000,3.0
19711,23341,268679909,23365,"2, Оба варианта",м. Солнцево (6 мин пешком),"Москва, Боровский проезд, 11",43.1,"5/5, Кирпичный",NaN,"50000.0 руб./ За месяц, Залог - 50000 руб., Ко...",+79653825192,Рассматриваются квартиранты только славяне и т...,Дизайнерский,NaN,Балкон (1),На улицу и двор,Совмещенный (1),Можно с детьми,"Мебель на кухне, Душевая каби

In [356]:
data['ceiling_height'].describe()

count    19696.000000
mean         2.861515
std          0.241997
min          1.200000
25%          2.600000
50%          3.000000
75%          3.000000
max          6.000000
Name: ceiling_height, dtype: float64

In [365]:
def get_metro_lamb(s):
    # Вспомогательная функция для извлечения метро
    return splfb(s,'м.','(').strip() if s is not None else None

def get_metro(data):
    # Функция для извлечения метро
    return data['Метро'].astype(str).apply(get_metro_lamb)

data['metro'] = get_metro(data)

/var/folders/l4/cnmt98l51vg5y92_6qkrhqq00000gn/T/ipykernel_41043/1271427225.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['metro'] = get_metro(data)


In [364]:
def get_metro_time_lamb(s):
    # Вспомогательная функция для извлечения времени до метро
    try:
        if 'пешком'in s:
            return float(splfb(s,'(',' мин').strip() if 'None' not in s else 0)*3 if s is not None or 'nan' else None
        else :
            return float(splfb(s,'(',' мин').strip() if 'None' not in s else 0) if s is not None or 'nan' else None
    except:
        pass

def get_metro_time(data):
    # Функция для извлечения времени до метро
    return data['Метро'].astype(str).apply(get_metro_time_lamb)

data['metro_time'] = get_metro_time(data)

/var/folders/l4/cnmt98l51vg5y92_6qkrhqq00000gn/T/ipykernel_41043/2167891232.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['metro_time'] = get_metro_time(data)


In [363]:
def get_adress_lamb(s):
    # Вспомогательная функция для извлечения адреса
    return s.split(',')[1]

def get_adress(data):
    # Функция для извлечения адреса
    return data['Адрес'].astype(str).apply(get_adress_lamb)


data['adress'] = get_adress(data)

/var/folders/l4/cnmt98l51vg5y92_6qkrhqq00000gn/T/ipykernel_41043/3381604294.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['adress'] = get_adress(data)


In [362]:
def get_area_lamb(s):
    # Вспомогательная функция для извлечения площади
    if '/'in s:
        return float(splbac(s,'/')[0])
    else:
        return float(s)

def get_area(data):
    # Функция для извлечения площади
    return data['Площадь, м2'].astype(str).apply(get_area_lamb)

data['full_area'] = get_area(data)

/var/folders/l4/cnmt98l51vg5y92_6qkrhqq00000gn/T/ipykernel_41043/4088508465.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['full_area'] = get_area(data)


In [366]:
data['full_area'].describe()

count    19696.000000
mean        63.240404
std         46.517018
min          7.000000
25%         38.900000
50%         49.300000
75%         69.000000
max        811.000000
Name: full_area, dtype: float64

In [367]:
def get_num_floors_lamb(s):
    # Вспомогательная функция для извлечения количества этажей
    if ',' in s:
        return float(s.split(',')[0].split('/')[1])
    else:
        return float(s.split('/')[1])

def get_num_floors(data):
    # Функция для извлечения количества этажей
    return data['Дом'].astype(str).apply(get_num_floors_lamb)

data['num_floors'] = get_num_floors(data)

/var/folders/l4/cnmt98l51vg5y92_6qkrhqq00000gn/T/ipykernel_41043/2754339417.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['num_floors'] = get_num_floors(data)


In [368]:
def get_floor_lamb(s):
    # Вспомогательная функция для извлечения этажа
    return float(s.split('/')[0])

def get_floor(data):
    # Функция для извлечения этажа
    return data['Дом'].astype(str).apply(get_floor_lamb)


data['floor'] = get_floor(data)

/var/folders/l4/cnmt98l51vg5y92_6qkrhqq00000gn/T/ipykernel_41043/2136882559.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['floor'] = get_floor(data)


In [369]:
def get_coord_y( data):
    # Функция для  определения координат по адресу

    with open('coordinates', 'rb') as file:
        addrdic = pickle.load(file)

    return  data['adress'].apply(lambda x: addrdic[x][0] if addrdic[x] is not None  else None )

def get_coord_x( data):
    # Функция для  определения координат по адресу

    with open('coordinates', 'rb') as file:
        addrdic = pickle.load(file)
    return  data['adress'].apply(lambda x: addrdic[x][1] if addrdic[x] is not None  else None )

data['coord_y'] = get_coord_x( data).clip(lower=36.74401, upper=38.140841)
data['coord_x'] = get_coord_y( data).clip(lower=55.386254, upper=56.119649)

data.head(10)

FileNotFoundError: [Errno 2] No such file or directory: 'coordinates'

In [384]:
def check_phrases(element):
    if isinstance(element, float):
        return 0
    if 'Мебель в комнатах' in element and 'Мебель на кухне' in element and 'Ванна' or 'Душ' in element and 'Стиральная машина' in element:
        return 1
    return 0


data['furnished'] = data['Дополнительно'].apply(check_phrases)

data.head(10)

/var/folders/l4/cnmt98l51vg5y92_6qkrhqq00000gn/T/ipykernel_41043/317350050.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['furnished'] = data['Дополнительно'].apply(check_phrases)


,index,ID объявления,Unnamed: 0,Количество комнат,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Телефоны,Описание,Ремонт,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление,col_rooms,price,ceiling_height,metro,metro_time,adress,full_area,num_floors,floor,Furnished,elevator,garbage_chute,repair,windows,balcony,bathroom,child_pet,furnished
0,0,271271157,0,4,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79166369231,Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.0,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157,4,500000,3.0,Смоленская,27.0,улица Новый Арбат,200.0,16.0,5.0,1,5,1,3,0,1,1,2,1
1,1,271634126,1,4,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79850608590,Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.5,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126,4,500000,3.5,Смоленская,24.0,улица Новый Арбат,198.0,16.0,5.0,1,2,0,3,2,1,3,1,1
2,2,271173086,2,"4, Оба варианта",м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...","+79672086536, +79099269384","ID 36380: Шикарная 4-х км. квартира в ЖК ""Нов...",Евроремонт,NaN,NaN,На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/271173086,4,500000,3.2,Смоленская,21.0,улица Новый Арбат,200.0,16.0,5.0,1,1,0,2,2,1,3,1,1
3,3,272197456,3,"4, Оба варианта",м. Смоленская (3 мин пешком),"Москва, переулок Плотников, 21С1",170.0/95.0/17.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...","+79660342340, +79099269384",ID 31618: Эксклюзивное предложение. Современн...,Евроремонт,14-42-20-19,NaN,На улицу и двор,Совмещенный (3),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/272197456,4,400000,3.2,Смоленская,9.0,переулок Плотников,170.0,6.0,5.0,1,1,0,2,2,1,3,1,1
4,4,273614615,4,2,м. Арбатская (7 мин пешком),"Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",NaN,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",+79852432860,Лот 111542. Татьяна Лучкина.\n\nБонус агенту 5...,Евроремонт,20 18,NaN,На улицу и двор,Совмещенный (2),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",The Book,NaN,3.9,"Пасс (1), Груз (1)",Да,https://www.cian.ru/rent/flat/273614615,2,225000,3.9,Арбатская,21.0,улица Новый Арбат,58.0,26.0,12.0,1,2,1,2,2,1,2,0,1
5,5,274837728,5,3,м. Смоленская (5 мин пешком),"Москва, 1-й Смоленский переулок, 21",92.0,3/7,NaN,"470000.0 руб./ За месяц, Залог - 470000 руб., ...",+79684728732,ID 743. С коллегами работаем 50/50. Видовые ап...,Дизайнерский,NaN,NaN,NaN,NaN,NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Smolensky De Luxe,NaN,NaN,NaN,NaN,https://www.cian.ru/rent/flat/274837728,3,470000,3.0,Смоленская,15.0,1-й Смоленский переулок,92.0,7.0,3.0,1,2,0,3,0,1,1,0,1
6,6,273643908,6,"5, Оба варианта",м. Арбатская (4 мин пешком),"Москва, переулок Романов, 3С1",213.0/140.0/20.0,4/5,наземная,"350000.0 руб./ За месяц, Залог - 350000 руб., ...","+79175135482, +79099269384",ID 41980: Предлагается 5-и комнатная квартира...,Евроремонт,43-25-25-25-22,NaN,На улицу и двор,Совмещенный (2),"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Сти...",NaN,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent

In [371]:
#Выводим в отдельную колонку общее число лифтов, вместо Nan у нас будет 0, потом мы это значение заполним средним по столбцу
data['elevator'] = data['Лифт'].astype(str).apply(lambda x: [int(num) for num in re.findall(r'\d+', x)])
data['elevator'] =  data['elevator'].apply(lambda x: sum(x))

/var/folders/l4/cnmt98l51vg5y92_6qkrhqq00000gn/T/ipykernel_41043/1176684279.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['elevator'] = data['Лифт'].astype(str).apply(lambda x: [int(num) for num in re.findall(r'\d+', x)])
/var/folders/l4/cnmt98l51vg5y92_6qkrhqq00000gn/T/ipykernel_41043/1176684279.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['elevator'] =  data['elevator'].apply(lambda x: sum(x))


In [372]:
#Все значения равные 0 мы заменяем на среднее по столбцу
mean_value =  data['elevator'][ data['elevator'] != 0].mean()
data['elevator'] =  data['elevator'].replace(0, mean_value).astype(int)

/var/folders/l4/cnmt98l51vg5y92_6qkrhqq00000gn/T/ipykernel_41043/1753569131.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['elevator'] =  data['elevator'].replace(0, mean_value).astype(int)


In [373]:
#Обработка колонки "Мусоропровод"
data['garbage_chute'] = data['Мусоропровод'].replace({'Да': 1, 'Нет': 0, np.nan: 0}).astype(int)
data.head(10)

/var/folders/l4/cnmt98l51vg5y92_6qkrhqq00000gn/T/ipykernel_41043/2600412347.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['garbage_chute'] = data['Мусоропровод'].replace({'Да': 1, 'Нет': 0, np.nan: 0}).astype(int)


,index,ID объявления,Unnamed: 0,Количество комнат,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Телефоны,Описание,Ремонт,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление,col_rooms,price,ceiling_height,metro,metro_time,adress,full_area,num_floors,floor,Furnished,elevator,garbage_chute
0,0,271271157,0,4,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79166369231,Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.0,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157,4,500000,3.0,Смоленская,27.0,улица Новый Арбат,200.0,16.0,5.0,1,5,1
1,1,271634126,1,4,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79850608590,Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.5,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126,4,500000,3.5,Смоленская,24.0,улица Новый Арбат,198.0,16.0,5.0,1,2,0
2,2,271173086,2,"4, Оба варианта",м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...","+79672086536, +79099269384","ID 36380: Шикарная 4-х км. квартира в ЖК ""Нов...",Евроремонт,NaN,NaN,На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/271173086,4,500000,3.2,Смоленская,21.0,улица Новый Арбат,200.0,16.0,5.0,1,1,0
3,3,272197456,3,"4, Оба варианта",м. Смоленская (3 мин пешком),"Москва, переулок Плотников, 21С1",170.0/95.0/17.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...","+79660342340, +79099269384",ID 31618: Эксклюзивное предложение. Современн...,Евроремонт,14-42-20-19,NaN,На улицу и двор,Совмещенный (3),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/272197456,4,400000,3.2,Смоленская,9.0,переулок Плотников,170.0,6.0,5.0,1,1,0
4,4,273614615,4,2,м. Арбатская (7 мин пешком),"Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",NaN,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",+79852432860,Лот 111542. Татьяна Лучкина.\n\nБонус агенту 5...,Евроремонт,20 18,NaN,На улицу и двор,Совмещенный (2),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",The Book,NaN,3.9,"Пасс (1), Груз (1)",Да,https://www.cian.ru/rent/flat/273614615,2,225000,3.9,Арбатская,21.0,улица Новый Арбат,58.0,26.0,12.0,1,2,1
5,5,274837728,5,3,м. Смоленская (5 мин пешком),"Москва, 1-й Смоленский переулок, 21",92.0,3/7,NaN,"470000.0 руб./ За месяц, Залог - 470000 руб., ...",+79684728732,ID 743. С коллегами работаем 50/50. Видовые ап...,Дизайнерский,NaN,NaN,NaN,NaN,NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Smolensky De Luxe,NaN,NaN,NaN,NaN,https://www.cian.ru/rent/flat/274837728,3,470000,3.0,Смоленская,15.0,1-й Смоленский переулок,92.0,7.0,3.0,1,2,0
6,6,273643908,6,"5, Оба варианта",м. Арбатская (4 мин пешком),"Москва, переулок Романов, 3С1",213.0/140.0/20.0,4/5,наземная,"350000.0 руб./ За месяц, Залог - 350000 руб., ...","+79175135482, +79099269384",ID 41980: Предлагается 5-и комнатная квартира...,Евроремонт,43-25-25-25-22,NaN,На улицу и двор,Совмещенный (2),"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Сти...",NaN,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/273643908,5,350000,3.2,Арбатская,12.0,переулок Романов,213.0,5.0,4.0,1,1,0
7,7,274475342,7,3,м. Смоленская (3 мин пешк

In [374]:
def process_repair_info(s):
    # Функция для обработки информации о ремонте
    if s == 'Без ремонта':
        return 0
    if pd.isna(s):
        return 1
    elif isinstance(s, str):
        if 'Дизайнерский' in s:
            return 3
        elif 'Евроремонт' in s:
            return 2
        elif 'Косметический' in s:
            return 1


# Применение функций к соответствующим столбцам данных
data['repair'] = data['Ремонт'].apply(process_repair_info)

/var/folders/l4/cnmt98l51vg5y92_6qkrhqq00000gn/T/ipykernel_41043/3383320241.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['repair'] = data['Ремонт'].apply(process_repair_info)


In [375]:
def process_window_info(s):
    if isinstance(s, str):
        if 'На улицу и двор' in s:
            return 2
        elif 'Во двор' in s:
            return 1
        elif 'На улицу' in s:
            return 0
    return 0

data['windows'] = data['Окна'].apply(process_window_info)

/var/folders/l4/cnmt98l51vg5y92_6qkrhqq00000gn/T/ipykernel_41043/1879866013.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['windows'] = data['Окна'].apply(process_window_info)


In [376]:
data['balcony'] = data['Балкон'].astype(str).apply(lambda x: [int(num) for num in re.findall(r'\d+', x)])
data['balcony'] =  data['balcony'].apply(lambda x: sum(x))

#Все значения равные 0 мы заменяем на среднее по столбцу
mean_value =  data['balcony'][ data['balcony'] != 0].mean()
data['balcony'] =  data['balcony'].replace(0, mean_value).astype(int)

/var/folders/l4/cnmt98l51vg5y92_6qkrhqq00000gn/T/ipykernel_41043/406014329.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['balcony'] = data['Балкон'].astype(str).apply(lambda x: [int(num) for num in re.findall(r'\d+', x)])
/var/folders/l4/cnmt98l51vg5y92_6qkrhqq00000gn/T/ipykernel_41043/406014329.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['balcony'] =  data['balcony'].apply(lambda x: sum(x))
/var/folders/l4/cnmt98l51vg5y92_6qkrhqq00000gn/T/ipykernel_41043/406014329.py:6: SettingWithC

In [377]:
data['bathroom'] = data['Санузел'].astype(str).apply(lambda x: [int(num) for num in re.findall(r'\d+', x)])
data['bathroom'] =  data['bathroom'].apply(lambda x: sum(x))

mean_value =  data['bathroom'][ data['bathroom'] != 0].mean()
data['bathroom'] =  data['bathroom'].replace(0, mean_value).astype(int)

/var/folders/l4/cnmt98l51vg5y92_6qkrhqq00000gn/T/ipykernel_41043/2528460388.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['bathroom'] = data['Санузел'].astype(str).apply(lambda x: [int(num) for num in re.findall(r'\d+', x)])
/var/folders/l4/cnmt98l51vg5y92_6qkrhqq00000gn/T/ipykernel_41043/2528460388.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['bathroom'] =  data['bathroom'].apply(lambda x: sum(x))
/var/folders/l4/cnmt98l51vg5y92_6qkrhqq00000gn/T/ipykernel_41043/2528460388.py:5: Setti

In [378]:
def process_children_pets_info(s):
    # Функция для обработки информации о детях и животных
    if pd.isna(s):
        return 0
    elif 'Можно с детьми, Можно с животными' in s:
        return 2
    elif 'Можно с детьми' in s:
        return 1
    elif 'Можно с животными' in s:
        return 1
    else:
        return 0

data['child_pet'] = data['Можно с детьми/животными'].apply(process_children_pets_info)
data.head(10)

/var/folders/l4/cnmt98l51vg5y92_6qkrhqq00000gn/T/ipykernel_41043/2648529950.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['child_pet'] = data['Можно с детьми/животными'].apply(process_children_pets_info)


,index,ID объявления,Unnamed: 0,Количество комнат,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Телефоны,Описание,Ремонт,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление,col_rooms,price,ceiling_height,metro,metro_time,adress,full_area,num_floors,floor,Furnished,elevator,garbage_chute,repair,windows,balcony,bathroom,child_pet
0,0,271271157,0,4,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79166369231,Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.0,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157,4,500000,3.0,Смоленская,27.0,улица Новый Арбат,200.0,16.0,5.0,1,5,1,3,0,1,1,2
1,1,271634126,1,4,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79850608590,Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.5,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126,4,500000,3.5,Смоленская,24.0,улица Новый Арбат,198.0,16.0,5.0,1,2,0,3,2,1,3,1
2,2,271173086,2,"4, Оба варианта",м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...","+79672086536, +79099269384","ID 36380: Шикарная 4-х км. квартира в ЖК ""Нов...",Евроремонт,NaN,NaN,На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/271173086,4,500000,3.2,Смоленская,21.0,улица Новый Арбат,200.0,16.0,5.0,1,1,0,2,2,1,3,1
3,3,272197456,3,"4, Оба варианта",м. Смоленская (3 мин пешком),"Москва, переулок Плотников, 21С1",170.0/95.0/17.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...","+79660342340, +79099269384",ID 31618: Эксклюзивное предложение. Современн...,Евроремонт,14-42-20-19,NaN,На улицу и двор,Совмещенный (3),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/272197456,4,400000,3.2,Смоленская,9.0,переулок Плотников,170.0,6.0,5.0,1,1,0,2,2,1,3,1
4,4,273614615,4,2,м. Арбатская (7 мин пешком),"Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",NaN,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",+79852432860,Лот 111542. Татьяна Лучкина.\n\nБонус агенту 5...,Евроремонт,20 18,NaN,На улицу и двор,Совмещенный (2),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",The Book,NaN,3.9,"Пасс (1), Груз (1)",Да,https://www.cian.ru/rent/flat/273614615,2,225000,3.9,Арбатская,21.0,улица Новый Арбат,58.0,26.0,12.0,1,2,1,2,2,1,2,0
5,5,274837728,5,3,м. Смоленская (5 мин пешком),"Москва, 1-й Смоленский переулок, 21",92.0,3/7,NaN,"470000.0 руб./ За месяц, Залог - 470000 руб., ...",+79684728732,ID 743. С коллегами работаем 50/50. Видовые ап...,Дизайнерский,NaN,NaN,NaN,NaN,NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Smolensky De Luxe,NaN,NaN,NaN,NaN,https://www.cian.ru/rent/flat/274837728,3,470000,3.0,Смоленская,15.0,1-й Смоленский переулок,92.0,7.0,3.0,1,2,0,3,0,1,1,0
6,6,273643908,6,"5, Оба варианта",м. Арбатская (4 мин пешком),"Москва, переулок Романов, 3С1",213.0/140.0/20.0,4/5,наземная,"350000.0 руб./ За месяц, Залог - 350000 руб., ...","+79175135482, +79099269384",ID 41980: Предлагается 5-и комнатная квартира...,Евроремонт,43-25-25-25-22,NaN,На улицу и двор,Совмещенный (2),"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Сти...",NaN,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/273643908,5,3500

In [379]:
#для получения координат по адресам
assert False
from geopy.geocoders import Nominatim
addarr= data['adress']
addrdic={}

def get_metro_station_coordinates(station_name):
    geolocator = Nominatim(user_agent="metro_locator")  

    # Добавим "метро" к запросу для более точного результата
    location = geolocator.geocode(f"{station_name} , Москва")

    if location:
        coordinates = (location.latitude, location.longitude)
        return coordinates
    else:
        print(f"Координаты для станции метро {station_name} не найдены.")
        return None

for i in addarr:
    try:
        if i in addrdic.keys():
            continue
        addrdic[i] = get_metro_station_coordinates(i)
    except:
        pass

with open('coordinates', 'wb') as file:
    pickle.dump(addrdic, file)



AssertionError: 

In [385]:
data_new = data.copy()
data_new

,index,ID объявления,Unnamed: 0,Количество комнат,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Телефоны,Описание,Ремонт,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление,col_rooms,price,ceiling_height,metro,metro_time,adress,full_area,num_floors,floor,Furnished,elevator,garbage_chute,repair,windows,balcony,bathroom,child_pet,furnished
0,0,271271157,0,4,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79166369231,Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.00,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157,4,500000,3.0,Смоленская,27.0,улица Новый Арбат,200.0,16.0,5.0,1,5,1,3,0,1,1,2,1
1,1,271634126,1,4,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79850608590,Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.50,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126,4,500000,3.5,Смоленская,24.0,улица Новый Арбат,198.0,16.0,5.0,1,2,0,3,2,1,3,1,1
2,2,271173086,2,"4, Оба варианта",м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...","+79672086536, +79099269384","ID 36380: Шикарная 4-х км. квартира в ЖК ""Нов...",Евроремонт,NaN,NaN,На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.20,Пасс (1),NaN,https://www.cian.ru/rent/flat/271173086,4,500000,3.2,Смоленская,21.0,улица Новый Арбат,200.0,16.0,5.0,1,1,0,2,2,1,3,1,1
3,3,272197456,3,"4, Оба варианта",м. Смоленская (3 мин пешком),"Москва, переулок Плотников, 21С1",170.0/95.0/17.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...","+79660342340, +79099269384",ID 31618: Эксклюзивное предложение. Современн...,Евроремонт,14-42-20-19,NaN,На улицу и двор,Совмещенный (3),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,3.20,Пасс (1),NaN,https://www.cian.ru/rent/flat/272197456,4,400000,3.2,Смоленская,9.0,переулок Плотников,170.0,6.0,5.0,1,1,0,2,2,1,3,1,1
4,4,273614615,4,2,м. Арбатская (7 мин пешком),"Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",NaN,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",+79852432860,Лот 111542. Татьяна Лучкина.\n\nБонус агенту 5...,Евроремонт,20 18,NaN,На улицу и двор,Совмещенный (2),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",The Book,NaN,3.90,"Пасс (1), Груз (1)",Да,https://www.cian.ru/rent/flat/273614615,2,225000,3.9,Арбатская,21.0,улица Новый Арбат,58.0,26.0,12.0,1,2,1,2,2,1,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19709,23339,215565511,23363,1,м. Говорово (8 мин пешком),"Москва, Боровское шоссе, 2к7, ш. Боровское (3 ...",35.0/16.4/8.0,10/14,NaN,"42000.0 руб./ За месяц, Залог - 42000 руб., Ко...",+79154383401,Сдаётся светлая теплая квартира (студия) с бал...,Евроремонт,NaN,Балкон (1),NaN,Совмещенный (1),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Сти...",Мещерский лес,NaN,NaN,"Пасс (1), Груз (1)",NaN,https://www.cian.ru/rent/flat/215565511,1,42000,3.0,Говорово,24.0,Боровское шоссе,35.0,14.0,10.0,1,2,0,2,0,1,1,0,1
19710,23340,274654844,23364,1,м. Солнцево (7 мин пешком),"Москва, Производственная улица, 8к1, ш. Боровс...",38.7/16.5/11.0,"5/18, Монолитный",NaN,"45000.0 руб./ За месяц, Залог - 45000 руб., Ко...",+79151173402,Сдам

In [386]:
data_new = data_new.drop(['Unnamed: 0', 'Количество комнат', 'Метро', 'Адрес', 'Площадь, м2', 'Дом', 'Парковка', 'Цена', 'Телефоны', 'Описание', 'Ремонт', 'Площадь комнат, м2', 'Балкон', 'Окна', 'Санузел', 'Можно с детьми/животными', 'Дополнительно', 'Название ЖК', 'Серия дома', 'Высота потолков, м', 'Лифт', 'Мусоропровод', 'Ссылка на объявление'], axis=1)
data_new

,index,ID объявления,col_rooms,price,ceiling_height,metro,metro_time,adress,full_area,num_floors,floor,Furnished,elevator,garbage_chute,repair,windows,balcony,bathroom,child_pet,furnished
0,0,271271157,4,500000,3.0,Смоленская,27.0,улица Новый Арбат,200.0,16.0,5.0,1,5,1,3,0,1,1,2,1
1,1,271634126,4,500000,3.5,Смоленская,24.0,улица Новый Арбат,198.0,16.0,5.0,1,2,0,3,2,1,3,1,1
2,2,271173086,4,500000,3.2,Смоленская,21.0,улица Новый Арбат,200.0,16.0,5.0,1,1,0,2,2,1,3,1,1
3,3,272197456,4,400000,3.2,Смоленская,9.0,переулок Плотников,170.0,6.0,5.0,1,1,0,2,2,1,3,1,1
4,4,273614615,2,225000,3.9,Арбатская,21.0,улица Новый Арбат,58.0,26.0,12.0,1,2,1,2,2,1,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19709,23339,215565511,1,42000,3.0,Говорово,24.0,Боровское шоссе,35.0,14.0,10.0,1,2,0,2,0,1,1,0,1
19710,23340,274654844,1,45000,3.0,Солнцево,21.0,Производственная улица,38.7,18.0,5.0,1,2,0,2,1,1,1,0,1
19711,23341,268679909,2,50000,3.0,Солнцево,18.0,Боровский проезд,43.1,5.0,5.0,1,2,0,3,2,1,1,1,1
19712,23342,274807525,2,55000,2.6,Солнцево,33.0,улица Богданова,52.5,23.0,8.0,1,3,1,2,1,1,2,0,1


In [387]:
data_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19696 entries, 0 to 19713
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   index           19696 non-null  int64  
 1   ID  объявления  19696 non-null  int64  
 2   col_rooms       19696 non-null  int64  
 3   price           19696 non-null  int64  
 4   ceiling_height  19696 non-null  float64
 5   metro           19696 non-null  object 
 6   metro_time      19351 non-null  float64
 7   adress          19696 non-null  object 
 8   full_area       19696 non-null  float64
 9   num_floors      19696 non-null  float64
 10  floor           19696 non-null  float64
 11  Furnished       19696 non-null  int64  
 12  elevator        19696 non-null  int64  
 13  garbage_chute   19696 non-null  int64  
 14  repair          19696 non-null  int64  
 15  windows         19696 non-null  int64  
 16  balcony         19696 non-null  int64  
 17  bathroom        19696 non-null 

In [389]:
data_new.describe()

,index,ID объявления,col_rooms,price,ceiling_height,metro_time,full_area,num_floors,floor,Furnished,elevator,garbage_chute,repair,windows,balcony,bathroom,child_pet,furnished
count,19696.000000,1.969600e+04,19696.000000,1.969600e+04,19696.000000,19351.000000,19696.000000,19696.000000,19696.000000,19696.00000,19696.000000,19696.000000,19696.000000,19696.000000,19696.000000,19696.000000,19696.00000,19696.00000
mean,11144.347939,2.671340e+08,1.997004,8.666193e+04,2.861515,26.501421,63.240404,14.493958,7.517973,0.85591,2.013911,0.520410,1.628199,0.771273,1.099157,1.194659,1.05854,0.85591
std,6552.037062,1.979105e+07,0.999843,1.282442e+05,0.241997,18.954661,46.517018,9.067100,5.992462,0.35119,1.160451,0.499596,0.726473,0.674889,0.345379,0.510548,0.74385,0.35119
min,0.000000,1.072986e+08,1.000000,1.200000e+04,1.200000,0.000000,7.000000,1.000000,-1.000000,0.00000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.00000,0.00000
25%,5528.750000,2.712438e+08,1.000000,3.999900e+04,2.600000,12.000000,38.900000,9.000000,3.000000,1.00000,2.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.00000,1.00000
50%,10970.500000,2.739377e+08,2.000000,5.000000e+04,3.000000,24.000000,49.300000,12.000000,6.000000,1.00000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.00000
75%,16714.250000,2.746976e+08,3.000000,7.500000e+04,3.000000,36.000000,69.000000,17.000000,10.000000,1.00000,2.000000,1.000000,2.000000,1.000000,1.000000,1.000000,2.00000,1.00000
max,23343.000000,2.750064e+08,6.000000,3.000000e+06,6.000000,357.000000,811.000000,116.000000,85.000000,1.00000,60.000000,1.000000,3.000000,2.000000,8.000000,6.000000,2.00000,1.00000
